In [1]:
#!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="DwQSgWGYxeQmRJ29BCTy")
project = rf.workspace("uni-vug0c").project("metal-film-leaded-resistor-color-bands")
dataset = project.version(4).download("yolov8")


Defaulting to user installation because normal site-packages is not writeable
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.160, to fix: `pip install ultralytics==8.0.134`

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Extracting Dataset Version Zip to Metal-Film-Leaded-Resistor-Color-Bands-4 in yolov8:: 100%|█| 1402/1402 [00:01<00:00, 


In [1]:
#!pip install ultralytics
from ultralytics import YOLO

# Load a model
model = YOLO("D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\YOLO_Segmenter\\runs\\segment\\train\\weights\\best.pt")

In [ ]:
# Train model with custom dataset
results = model.train(data="Metal-Film-Leaded-Resistor-Color-Bands-4\\data.yaml", epochs=100)

In [5]:
version = project.version(4)
version.deploy("yolov8", "D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\runs\\segment\\train3\\")

Dependency ultralytics==8.0.134 is required but found version=8.0.160, to fix: `pip install ultralytics==8.0.134`
View the status of your deployment at: https://app.roboflow.com/uni-vug0c/metal-film-leaded-resistor-color-bands/4
Share your model with the world at: https://universe.roboflow.com/uni-vug0c/metal-film-leaded-resistor-color-bands/model/4


In [4]:
import os
def get_files(dir):
    tests = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            tests.append(os.path.join(dir,name))
    return tests

files = get_files("D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\Tests")
path_test = 'D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\Tests\\crop.jpg'

In [41]:
pred = model(path_test)


image 1/1 D:\Joule\Documents\Jhony\Universidade\UTFPR\2023.2\Oficinas\Tests\crop.jpg: 288x640 4 Resistor-Color-Bandss, 1 Resistor-Color-Bands-Start, 153.0ms
Speed: 11.0ms preprocess, 153.0ms inference, 10.0ms postprocess per image at shape (1, 3, 288, 640)


In [58]:
print(pred[0].masks.xy[1][0])

[     116.16      15.981]


In [4]:
import cv2 as cv
from PIL import Image

for file in files:
    npath = "\\".join(file.split("\\")[:-2])  # Gets root dir
    npath = os.path.join(npath, "Resistores_Cropped\\"+file.split("\\")[-1])
    if not os.path.isfile(npath):
        try:
            pred = model([file])
            img = cv.imread(file)
            box = [int(i) for i in pred[0].boxes.data[0]]
            img = img[box[1]:box[3], box[0]:box[2]]
            Image.fromarray(cv.cvtColor(img, cv.COLOR_BGR2RGB)).save(npath)
        except IndexError:
            pass



0: 384x640 (no detections), 98.5ms
Speed: 3.0ms preprocess, 98.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.1ms
Speed: 1.0ms preprocess, 8.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.1ms
Speed: 1.0ms preprocess, 9.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.0ms
Speed: 1.9ms preprocess, 8.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.5ms
Speed: 2.0ms preprocess, 9.5ms inference, 1.

In [5]:
import numpy as np
from numpy.linalg import norm

def vect_angle(color1, color2):
    dp = np.dot(color1, color2)/(norm(color1)*norm(color2))
    if dp > 1:
        dp = 1.0
    elif dp < -1:
        dp = -1.0
    return np.arccos(dp)  # in radians

def get_neighbours(imarr, r, c, kernel_size=3):
    """
    """
    neighbours = list()
    k_s = kernel_size//2
    for i in range(r-k_s, r+k_s+1):
        for j in range(c-k_s, c+k_s+1):
            if i >= 0 and i < len(imarr) and j >= 0 and j < len(imarr[i]):
                if (r, c) != (i, j):
                    neighbours.append(imarr[i][j])
    return neighbours

def filter_neighbours(pixel, neighbours, angle_thresh, dist_thresh):
    """
    """
    valid = list()
    for neighbour in neighbours:
        if get_angle(pixel, neighbour) < angle_thresh and (get_module(pixel) - get_module(neighbour)) < dist_thresh:
            valid.append(neighbour)
    return valid

def scalar_prod(vector, scalar): 
    return [i*scalar for i in vector]

def vector_sum(v1, v2):
    return [a+b for a,b in zip(v1, v2)]

def sum_vectors(lv):
    if len(lv) == 1:
        return lv[0]
    elif len(lv) == 0:
        raise IndexError
    rs = vector_sum(lv[0], lv[1])
    for i in lv[2:]:
        rs = vector_sum(rs, i)
    return rs

def color_selective_blur(imarr, retention_rate, thresh=5):
    """
    Acts like a gaussian blur (center of Kernel is equal to retention_rate here), but only takes
    into consideration the pixels that are thresh away in modulus (normalized vector distances)
    INPUTS
        imarr: 2D array
        retention_rate: 0.0-1.0
    """
    for row in range(len(imarr)):
        for column in range(len(imarr[row])):
            neighbours = get_neighbours(imarr, row, column)
            neighbours = filter_neighbours(imarr[row][column], neighbours, 3.1415926535/24, 100)
            if len(neighbours):
                weight = (1-retention_rate)/len(neighbours)
            else:
                neighbours = get_neighbours(imarr, row, column)
                weight = (1-retention_rate)/len(neighbours)
            neighbours = [scalar_prod(i, weight) for i in neighbours]
            somm = sum_vectors(neighbours)
            imarr[row][column] = vector_sum(scalar_prod(imarr[row][column], retention_rate), somm)

In [5]:
from PIL import Image
import cv2 as cv
Colour_Range = [
    [(0, 0, 0), (255, 255, 20), "BLACK", 0, (0, 0, 0)],
    [(0, 90, 10), (15, 250, 100), "BROWN", 1, (0, 51, 102)],
    [(0, 30, 80), (10, 255, 200), "RED", 2, (0, 0, 255)],
    [(5, 150, 150), (15, 235, 250), "ORANGE", 3, (0, 128, 255)],  # ok
    [(50, 100, 100), (70, 255, 255), "YELLOW", 4, (0, 255, 255)],
    [(45, 100, 50), (75, 255, 255), "GREEN", 5, (0, 255, 0)],  # ok
    [(100, 150, 0), (140, 255, 255), "BLUE", 6, (255, 0, 0)],  # ok
    [(120, 40, 100), (140, 250, 220), "VIOLET", 7, (255, 0, 127)],
    [(0, 0, 50), (179, 50, 80), "GRAY", 8, (128, 128, 128)],
    [(0, 0, 90), (179, 15, 250), "WHITE", 9, (255, 255, 255)],
]

# Bilateral filter and resize the image
img = cv.imread('Tests\\resistor_iluminado_crop.png')
def bilateral_filter(img):
    D, I = 4, 150
    scale = 8
    
    fimg = cv.bilateralFilter(img, D, I, I)
    width = fimg.shape[1]*scale
    height = fimg.shape[0]*scale
    fimg = cv.resize(fimg, (width, height))
    return fimg

#Image.fromarray(cv.cvtColor(fimg, cv.COLOR_RGB2BGR)).save(f"Tests\\bilateral_filter_2_D{D}_I{I}.png")


In [21]:
from copy import deepcopy
from PIL import Image
import cv2 as cv
import numpy as np
from numpy.linalg import norm

def dot_product(v1, v2):
    return sum(int(a)*int(b) for a,b in zip(v1, v2))

def vect_angle(color1, color2):
    dp = dot_product(color1, color2)/(norm(color1)*norm(color2))
    if dp > 1:
        dp = 1.0
    elif dp < -1:
        dp = -1.0
    return np.arccos(dp)  # in radians

def bilateral_filter(img):
    D, I = 4, 150
    target_width = 100
    scale = target_width/img.shape[1]
    #
    fimg = cv.bilateralFilter(img, D, I, I)
    width = fimg.shape[1]*scale
    height = fimg.shape[0]*scale
    fimg = cv.resize(fimg, (int(width), int(height)))
    return fimg

class Contour:
    def __init__(self, points=[]):
        self.points = list(points)
    # Adds data and/or updates Contour boundaries
    def update(self, points=[]):
        self.points.append(i for i in points)
    # Calculates and returns the sub-boundaries in self.points
    def get_boundaries(self):
        # Returns neighbors to a point dist-pixels away
        def get_neighbors(point, dist=3):
            n = list()
            for i in range(-(dist//2),1+dist//2):
                for j in range(-(dist//2),1+dist//2):
                    n.append(list(np.add(point, (i,j))))
            return n
        #
        def in_boundary(point, boundary):
            if point >= boundary[0] and point < boundary[1]:
                return True
            else:
                return False
        # Iteratively finds sub-boundaries
        subb = []
        points = deepcopy(self.points)
        for point in self.points:
            contained = False
            index = -1
            for i in range(len(subb)):
                if not contained and in_boundary(point, subb[i]):
                    contained = True
                    index = i
                    break
            if not contained:
                subb.append([point, point])
            #
            neighbors = [n for n in get_neighbors(point) if (list(n) in self.points or tuple(n) in self.points)]
            # Updates boundary with top-left-most and bottom-right-most points in its proximity
            subb[index][0] = min(list(min(neighbors)), list(subb[index][0]))
            subb[index][1] = max(list(max(neighbors)), list(subb[index][1]))
        return subb            
    # Returns the index-th boundary
    def __getitem__(self, index):
        return self.points[index]


class Color:
    def __init__(self, image, pos, thresh = 255, angle_thresh = np.pi*15/180):
        self.image = image
        self.color = image[pos[0]][pos[1]]
        self.coords = [pos]  # List of the coordinates of the pixels that have the 'same' color
        self.thresh = thresh
        self.angle_thresh = angle_thresh
    # Overloads the equality operator (==)
    def __eq__(self, pos):
        c = self.image[pos[0]][pos[1]]
        if abs(norm(self.color) - norm(c)) <= self.thresh and vect_angle(self.color, c) < self.angle_thresh:
            return True
        else:
            return False
    # Overloads the contains operator (in)
    def __contains__(self, pos):
        return self.__eq__(pos)
    # Adds a color to the group
    def append(self, pos):
        if pos not in self.coords:
            self.coords.append(pos)
        R,G,B = 0,0,0
        for x,y in self.coords:
            R += self.image[x][y][0]
            G += self.image[x][y][1]
            B += self.image[x][y][2]
        R /= len(self.coords)
        G /= len(self.coords)
        B /= len(self.coords)
        self.color = [R,G,B]

class Group:
    def __init__(self, objs = []):
        self.objects = objs
    # Adds an object to the group
    def insert(self, obj):
        self.objects.append(obj)
    # Checks if an element is in any of its subgroups
    def __contains__(self, elem):
        for obj in self.objects:
            if elem in obj:
                return True
        return False
    # Adds an element to one of its subgroups
    def append(self, elem):
        for obj in self.objects:
            if elem in obj:
                obj.append(elem)
                return

def find_color_pools(im, color_space=cv.COLOR_BGR2HSV):
    """
    """
    img = cv.cvtColor(im, color_space)
    groups = Group()
    for i in range(len(img)):
        for j in range(len(img[i])):
            if (i,j) not in groups:
                cg = Color(img, (i,j))
                groups.insert(cg)
            else:
                groups.append([i,j])
    return groups

# Paints similar colors the same
"""for file in get_files(os.path.join(os.getcwd(), "Tests\\light")):
    img = cv.imread(file)
    img = bilateral_filter(img)
    groups = find_color_pools(img)
    imgcp = deepcopy(img)
    for obj in groups.objects:
        for point in obj.coords:
            imgcp[point[0]][point[1]] = obj.color
    #
    Image.fromarray(cv.cvtColor(imgcp, cv.COLOR_HSV2RGB)).save(os.path.join(os.getcwd(), f"{'.'.join(file.split('.')[:-1])}_colorized.png"))
"""
file = "Tests\\resistor_iluminado_crop.png"
img = cv.imread(file)
img = bilateral_filter(img)
groups = find_color_pools(img)
imgcp = deepcopy(img)
for obj in groups.objects:
    for point in obj.coords:
        imgcp[point[0]][point[1]] = obj.color
#
Image.fromarray(cv.cvtColor(imgcp, cv.COLOR_HSV2RGB)).save(os.path.join(os.getcwd(), f"{'.'.join(file.split('.')[:-1])}_colorized.png"))
"""
path = "D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\Tests\\resistor_iluminado_crop.png"
img = cv.imread(path)
img = bilateral_filter(img)
groups = find_color_pools(img)
imgcp = deepcopy(img)
sizes = [len(i.coords) for i in groups.objects]
sizes.sort(reverse=True)
for obj in groups.objects:
    for size in sizes[:2]:
        if len(obj.coords) < size:
            c = Contour(obj.coords)
            print(c.get_boundaries())
#
#Image.fromarray(cv.cvtColor(imgcp, cv.COLOR_HSV2RGB)).save(os.path.join(os.getcwd(), f"{'.'.join(file.split('.')[:-1])}_colorized.png"))
"""

C:\Users\jhony\AppData\Local\Temp\ipykernel_1368\1071019581.py:24: RuntimeWarning: overflow encountered in scalar subtract
  H = 4 + 60*(color[2] - color[1])/(M - m)
C:\Users\jhony\AppData\Local\Temp\ipykernel_1368\1071019581.py:30: RuntimeWarning: overflow encountered in scalar subtract
  H = 60*(color[1] - color[0])/(M - m)


'\npath = "D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\Tests\\resistor_iluminado_crop.png"\nimg = cv.imread(path)\nimg = bilateral_filter(img)\ngroups = find_color_pools(img)\nimgcp = deepcopy(img)\nsizes = [len(i.coords) for i in groups.objects]\nsizes.sort(reverse=True)\nfor obj in groups.objects:\n    for size in sizes[:2]:\n        if len(obj.coords) < size:\n            c = Contour(obj.coords)\n            print(c.get_boundaries())\n#\n#Image.fromarray(cv.cvtColor(imgcp, cv.COLOR_HSV2RGB)).save(os.path.join(os.getcwd(), f"{\'.\'.join(file.split(\'.\')[:-1])}_colorized.png"))\n'

In [22]:
len(groups.objects)

1021